<img src="img/csdms_logo.jpg">

# Visualization with `ipywidgets`

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact
from heat_solver import BmiHeat

Define a function that runs the `HeatSolver` model a fixed number of time steps. Store the temperture field from each time step in a list. Return the list.

In [ ]:
def run_model(n_steps=10):
    heat = BmiHeat()
    heat.initialize('../heat.yaml')
    
    initial_temp = heat.get_value('plate_surface__temperature')
    initial_temp[2, 2] = 100.
    heat.set_value('plate_surface__temperature', initial_temp)
    
    temp = [initial_temp]
    for _ in range(n_steps):
        heat.update()
        new_temp = heat.get_value('plate_surface__temperature')
        temp.append(new_temp)
    
    heat.finalize()
    return temp

Run the model for awhile.

In [ ]:
temperature = run_model(50)

Define a function to visualize the temperature field at a time step (defined by an index into the list of temperature fields).

In [ ]:
def plot_temperature(time_index=0):
    xmin, xmax = 0., temperature[0].shape[1]
    ymin, ymax = 0., temperature[0].shape[0]
    plt.imshow(temperature[time_index], extent=[xmin, xmax, ymin, ymax], origin='upper', cmap='Reds')
    plt.title('Plate Temperature')
    plt.colorbar().ax.set_ylabel('Temperature (K)')

Make an interactive plot of temperature over time.

In [ ]:
interact(plot_temperature, time_index=(0, 50, 1))

## Update the model interactively

Define a function to plot a 2D array of temperatures.

In [ ]:
def plot_temperature(temperature):
    xmin, xmax = 0., temperature.shape[1]
    ymin, ymax = 0., temperature.shape[0]
    plt.imshow(temperature, extent=[xmin, xmax, ymin, ymax], origin='upper', cmap='Reds')
    plt.title('Plate Temperature')
    plt.colorbar().ax.set_ylabel('Temperature (K)')

We're going to use this a little later to interactively run and plot our model.

Create an object that runs the `BmiHeat` model until a given time and then plots the temperature field at that time. Notice that this would work with any BMI model - you would just have to change the name of the variable to plot. Since our model cannot run backward in time, we cache the temperature at each time step.

In [ ]:
class HeatRunner(object):
    def __init__(self, heat):
        self._heat = heat
        self._cache = []

    def run_and_plot(self, time_index):
        for _ in range(len(self._cache), time_index):
            self._heat.update()
            self._cache.append(heat.get_value('plate_surface__temperature'))
        plot_temperature(self._cache[time_index - 1])

Instantiate the heat model and initialize it.

In [ ]:
heat = BmiHeat()
heat.initialize('../heat.yaml')

Set the initial temperature.

In [ ]:
init_temperature = heat.get_value('plate_surface__temperature')
init_temperature[2, 2] = 100.
heat.set_value('plate_surface__temperature', init_temperature)

Run the model interactively over time. Note that as you move the slider, the model *runs* to the updated time.

In [ ]:
runner = HeatRunner(heat)
interact(runner.run_and_plot, time_index=(0, 100, 1))

Get a reference to the current temperature of the plate...

In [ ]:
current_temperature = heat.get_value_ref('plate_surface__temperature')

... and update the current temperature as you like and the result will be reflected above as you continue to advance the slider.

In [ ]:
current_temperature[2, 8] = 5 * current_temperature.max()